In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets scikit-learn pandas tqdm

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_

In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cpu


In [3]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Change this to your desired checkpoint
checkpoint_path = "/content/drive/MyDrive/imdb-bert1/checkpoint-epoch3-fine-tuned-fine-tuned-fine-tuned"


tokenizer = BertTokenizer.from_pretrained(checkpoint_path)
model = BertForSequenceClassification.from_pretrained(checkpoint_path)
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [4]:
def predict_sentiment(text):
    # Tokenize
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Predict
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        pred = torch.argmax(probs, dim=1).item()
        label = "Agriculture" if pred == 1 else "Non-agriculture"
        confidence = probs[0][pred].item()

    return label, confidence


In [11]:
predict_sentiment("pest control method")

('Agriculture', 0.9999325275421143)

In [7]:
examples = [
    "Bhai kal movie chalna hai kya?",
    "Yeh phone camera best hai boss!",
    "Modiji ne speech diya aaj Parliament mein.",
    "Kal midsem exam hai aur kuchh bhi nahi padha.",
    "Sahni is great.",
    "I have a meeting with the Prime Minister tomorrow.",
    "Bhai code compile nahi ho raha, help!",
    "Woh ladki sach mein intelligent hai.",
    "Yaar kal match miss kar diya maine.",
    "Kaunsa game khel raha hai tu aajkal?",
    "Meri prime minister ke saath meeting hai.",
    "Aaloo ki fasal kaise seechein",
    "Meri ummeed se behtar kaam kiya hai bhai tune.",
    "Aaloo khaao sugar badhaao."
]


for text in examples:
    label, conf = predict_sentiment(text)
    print(f"📝 Review: {text}\n🔎 Prediction: {label} ({conf*100:.2f}% confidence)\n")


📝 Review: Bhai kal movie chalna hai kya?
🔎 Prediction: Non-agriculture (99.00% confidence)

📝 Review: Yeh phone camera best hai boss!
🔎 Prediction: Non-agriculture (99.99% confidence)

📝 Review: Modiji ne speech diya aaj Parliament mein.
🔎 Prediction: Non-agriculture (99.97% confidence)

📝 Review: Kal midsem exam hai aur kuchh bhi nahi padha.
🔎 Prediction: Non-agriculture (99.97% confidence)

📝 Review: Sahni is great.
🔎 Prediction: Non-agriculture (99.96% confidence)

📝 Review: I have a meeting with the Prime Minister tomorrow.
🔎 Prediction: Non-agriculture (99.35% confidence)

📝 Review: Bhai code compile nahi ho raha, help!
🔎 Prediction: Non-agriculture (99.96% confidence)

📝 Review: Woh ladki sach mein intelligent hai.
🔎 Prediction: Non-agriculture (99.97% confidence)

📝 Review: Yaar kal match miss kar diya maine.
🔎 Prediction: Non-agriculture (99.99% confidence)

📝 Review: Kaunsa game khel raha hai tu aajkal?
🔎 Prediction: Non-agriculture (91.73% confidence)

📝 Review: Meri prime mi

In [ ]:
# Case A — individual string
label1, conf1 = predict_sentiment("Sahni is great.")
print(f"A: {label1} ({conf1*100:.2f}%)")

# Case B — inside a list loop
for text in ["Sahni is great."]:
    label2, conf2 = predict_sentiment(text)
    print(f"B: {label2} ({conf2*100:.2f}%)")


A: Non-agriculture (96.35%)
B: Non-agriculture (96.35%)


In [12]:
examples = [
    "Wheat crops are drying even though the soil has enough moisture",  # No punctuation
    "Wheat crops are drying even though the soil has enough moisture.",  # With period

    "There is pest attack in my tomato plants",
    "There is pest attack in my tomato plants.",

    "Can I use neem oil for aphid control in okra",
    "Can I use neem oil for aphid control in okra.",

    "My sugarcane fields are turning yellow suddenly",
    "My sugarcane fields are turning yellow suddenly.",

    "Is drip irrigation suitable for sandy soil",
    "Is drip irrigation suitable for sandy soil.",
]

for text in examples:
    label, conf = predict_sentiment(text)
    print(f"📝 Review: {text}\n🔎 Prediction: {label} ({conf*100:.2f}% confidence)\n")


📝 Review: Wheat crops are drying even though the soil has enough moisture
🔎 Prediction: Agriculture (99.94% confidence)

📝 Review: Wheat crops are drying even though the soil has enough moisture.
🔎 Prediction: Agriculture (99.91% confidence)

📝 Review: There is pest attack in my tomato plants
🔎 Prediction: Agriculture (99.97% confidence)

📝 Review: There is pest attack in my tomato plants.
🔎 Prediction: Agriculture (99.96% confidence)

📝 Review: Can I use neem oil for aphid control in okra
🔎 Prediction: Agriculture (99.99% confidence)

📝 Review: Can I use neem oil for aphid control in okra.
🔎 Prediction: Agriculture (99.98% confidence)

📝 Review: My sugarcane fields are turning yellow suddenly
🔎 Prediction: Agriculture (98.11% confidence)

📝 Review: My sugarcane fields are turning yellow suddenly.
🔎 Prediction: Agriculture (98.47% confidence)

📝 Review: Is drip irrigation suitable for sandy soil
🔎 Prediction: Agriculture (99.98% confidence)

📝 Review: Is drip irrigation suitable for sa

In [13]:
examples = [
    # 1. Real agriculture, informal
    "Bhaiya meri chilli ki kheti me kuchh sukh raha hai samajh nahi aa raha",
    "Bhaiya meri chilli ki kheti me kuchh sukh raha hai samajh nahi aa raha.",

    # 2. Real agriculture
    "Tomato me safed dabbey pad gaye hai kya karun",
    "Tomato me safed dabbey pad gaye hai kya karun.",

    # 3. Vague
    "I am best",
    "I am best.",

    # 4. Semi-agriculture, informal
    "Main kal farm gaya tha but kuchh naya fertilizer mila",
    "Main kal farm gaya tha but kuchh naya fertilizer mila.",

    # 5. Real agriculture
    "Gaon me paani ki dikkat hai kheti kaise ho",
    "Gaon me paani ki dikkat hai kheti kaise ho.",

    # 6. Real agriculture
    "Aaj garmi bhut hai lagta hai crops bhi suffer kar rahe",
    "Aaj garmi bhut hai lagta hai crops bhi suffer kar rahe.",

    # 7. Personal/vague
    "Bhai sahni is great",
    "Bhai sahni is great.",

    # 8. Real agriculture
    "Ye spray kaunsa lena hai for green chilli",
    "Ye spray kaunsa lena hai for green chilli.",

    # 9. Real agriculture
    "Meri field me podhe gir rahe hai lagta hai wind se",
    "Meri field me podhe gir rahe hai lagta hai wind se.",

    # 10. Borderline
    "Just had maggi and chai in farm",
    "Just had maggi and chai in farm.",

    # 11. Misleading with word “farm”
    "Kya cricket khelne chalein farm ke piche",
    "Kya cricket khelne chalein farm ke piche.",

    # 12. Real agriculture
    "Pesticide ka result nahi dikh raha",
    "Pesticide ka result nahi dikh raha.",
]


for text in examples:
    label, conf = predict_sentiment(text)
    print(f"📝 Review: {text}\n🔎 Prediction: {label} ({conf*100:.2f}% confidence)\n")



📝 Review: Bhaiya meri chilli ki kheti me kuchh sukh raha hai samajh nahi aa raha
🔎 Prediction: Agriculture (63.31% confidence)

📝 Review: Bhaiya meri chilli ki kheti me kuchh sukh raha hai samajh nahi aa raha.
🔎 Prediction: Non-agriculture (73.87% confidence)

📝 Review: Tomato me safed dabbey pad gaye hai kya karun
🔎 Prediction: Agriculture (99.33% confidence)

📝 Review: Tomato me safed dabbey pad gaye hai kya karun.
🔎 Prediction: Agriculture (98.00% confidence)

📝 Review: I am best
🔎 Prediction: Non-agriculture (98.45% confidence)

📝 Review: I am best.
🔎 Prediction: Non-agriculture (99.30% confidence)

📝 Review: Main kal farm gaya tha but kuchh naya fertilizer mila
🔎 Prediction: Agriculture (99.92% confidence)

📝 Review: Main kal farm gaya tha but kuchh naya fertilizer mila.
🔎 Prediction: Agriculture (99.89% confidence)

📝 Review: Gaon me paani ki dikkat hai kheti kaise ho
🔎 Prediction: Agriculture (97.13% confidence)

📝 Review: Gaon me paani ki dikkat hai kheti kaise ho.
🔎 Prediction

In [ ]:
import pandas as pd
import os

# Step 1: Create CSV file if it doesn't exist
file_path = "fine_tune_patch.csv"

if not os.path.exists(file_path):
    df = pd.DataFrame(columns=["text", "label"])
    df.to_csv(file_path, index=False)
    print("✅ Created new file: fine_tune_patch.csv")
else:
    print("📁 File already exists, will append new rows.")

# Step 2: Paste your DeepSeek batch here (example batch)
raw_data = """
text,label
Just got a new phone its amazing,0
The new cafe downtown has the best coffee,0
Can anyone recommend a good gym in the area,0
Traffic was horrible on my way to work today,0
I love watching sci fi movies on weekends,0
Learning to play guitar is harder than I thought,0
Does anyone know a good mechanic for car repairs,0
Online shopping is so convenient these days,0
My laptop crashed and I lost all my files,0
Which is the best travel destination in Europe,0
Studying for exams is so stressful right now,0
Netflix has some great new shows this month,0
Has anyone tried the new pizza place in town,0
Working from home has its pros and cons,0
I need a good hairstylist for a haircut,0
Traffic jams are the worst during rush hour,0
Which smartphone should I buy under 30k,0
Listening to music helps me relax after work,0
Does anyone know a good dentist nearby,0
Summer heat is unbearable without AC,0
Planning a trip to Goa next month,0
Electric cars are the future of transportation,0
Kal raat party bhot mast thi bhai,0
New restaurant me biryani ka taste acha nahi hai,0
Gym join karna chahiye for weight loss,0
Office ka traffic itna badh gaya hai,0
Movie dekhne ka plan banao weekend pe,0
Guitar bajana seekh raha hu par mushkil hai,0
Car repair ke liye koi accha mechanic chahiye,0
Online shopping se time bachta hai,0
Laptop hang ho gaya saara data lost hogaya,0
Europe trip ke liye best place konsa hai,0
Exams ki tension se neend nahi aa rahi,0
Netflix pe naye shows aaye hai dekhna hai,0
New pizza outlet try kiya kaisa hai,0
Work from home me distractions bhot hote hai,0
Haircut ke liye koi accha salon batao,0
Rush hour me traffic se bachne ka koi tarika,0
30k budget me konsa smartphone lena chahiye,0
Music sunke stress kam hota hai,0
Dentist ka appointment lena hai koi accha,0
Bina AC ke garmi seh nahi ja rahi,0
Goa trip ka plan kar rahe hai next month,0
Electric cars ab future hai petrol khatam,0
"""

# Step 3: Convert raw_data string to DataFrame
from io import StringIO

new_df = pd.read_csv(StringIO(raw_data), header=None, names=["text", "label"])

# Step 4: Append to existing CSV
existing_df = pd.read_csv(file_path)
combined_df = pd.concat([existing_df, new_df], ignore_index=True)
combined_df.to_csv(file_path, index=False)
print(f"✅ Added {len(new_df)} rows to fine_tune_patch.csv")


📁 File already exists, will append new rows.
✅ Added 45 rows to fine_tune_patch.csv


In [ ]:
import pandas as pd

# Load CSV while keeping header
file_path = "fine_tune_patch.csv"
df = pd.read_csv(file_path)

# Drop any rows after the first where text == 'text' and label == 'label'
df_cleaned = df[~((df.index != 0) & (df['text'] == 'text') & (df['label'] == 'label'))]

# Save cleaned file
df_cleaned.to_csv(file_path, index=False)
print(f"✅ Cleaned file saved with {len(df_cleaned)} rows (excluding accidental headers).")


✅ Cleaned file saved with 271 rows (excluding accidental headers).


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import torch
import pandas as pd

# Load model & tokenizer
model = BertForSequenceClassification.from_pretrained(checkpoint_path)
tokenizer = BertTokenizer.from_pretrained(checkpoint_path)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# Load your correction dataset
df = pd.read_csv("fine_tune_patch.csv")

# Convert the 'label' column to integers, coercing errors
df['label'] = pd.to_numeric(df['label'], errors='coerce')

# Drop rows where 'label' could not be converted to a number (e.g., the header row)
df.dropna(subset=['label'], inplace=True)

# Convert labels to integers
labels = df['label'].astype(int).tolist()
texts = df['text'].tolist()


# Tokenize
encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)

# Dataset class
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }
    def __len__(self):
        return len(self.labels)

dataset = CustomDataset(encodings, labels)
loader = DataLoader(dataset, batch_size=4, shuffle=True)

In [ ]:
from torch.optim import AdamW
from tqdm import tqdm  # ✅ Progress bar

optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 3

for epoch in range(num_epochs):
    print(f"🚀 Epoch {epoch+1}/{num_epochs}")
    model.train()
    loop = tqdm(loader, desc=f"Epoch {epoch+1}", unit="batch")

    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        loop.set_postfix(loss=loss.item())

    print(f"✅ Epoch {epoch+1} completed.\n")

# Save the fine-tuned model
model.save_pretrained(f"{checkpoint_path}-fine-tuned")
tokenizer.save_pretrained(f"{checkpoint_path}-fine-tuned")
print("✅ Model saved after fine-tuning.")


🚀 Epoch 1/3


Epoch 1: 100%|██████████| 68/68 [02:25<00:00,  2.15s/batch, loss=0.0314]


✅ Epoch 1 completed.

🚀 Epoch 2/3


Epoch 2: 100%|██████████| 68/68 [02:18<00:00,  2.04s/batch, loss=0.000481]


✅ Epoch 2 completed.

🚀 Epoch 3/3


Epoch 3: 100%|██████████| 68/68 [02:18<00:00,  2.04s/batch, loss=0.00344]


✅ Epoch 3 completed.

✅ Model saved after fine-tuning.


In [14]:

import gradio as gr
import torch

# Function to predict label
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        pred = torch.argmax(probs, dim=1).item()
        label = "Agriculture" if pred == 1 else "Non-agriculture"
        confidence = probs[0][pred].item()
        return f"{label} ({confidence*100:.2f}%)"

# Gradio UI
demo = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(lines=4, placeholder="Enter your text here..."),
    outputs="text",
    title="🌾 Agriculture Detector",
    description="Paste your text below and find out whether it's agriculture-related or not."
)

# Launch
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8272a15648601d6204.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
